In [1]:
import json
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML

# Pad naar de bestanden
food_file_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Food.json"
content_file_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Content.json"

# Food ID instellen waarop gefilterd moet worden
selected_food_id = 506  # Vervang dit nummer met het gewenste food_id

C:\Users\bramd_finhsgu\AppData\Local\Temp\ipykernel_25664\4084231479.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [2]:
# Animal foods filteren om id's te bekomen
filtered_food_data = []
with open(food_file_path, "r", encoding="utf-8") as food_file:
    for line in food_file:
        try:
            item = json.loads(line.strip())
            if item.get("food_group") == "Animal foods":
                filtered_food_data.append({"id": item.get("id"), "name": item.get("name")})
        except json.JSONDecodeError as e:
            print(f"Fout bij het verwerken van een regel in food.json: {e}")

food_df = pd.DataFrame(filtered_food_data)

food_output_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\id_name_foodb.csv"
food_df.to_csv(food_output_path, index=False)
print(f"\nFood.json DataFrame opgeslagen als: {food_output_path}")

# Eerste paar regels van de DataFrame weergeven
print("Food.json eerste paar regels:")
print(food_df.head())


Food.json DataFrame opgeslagen als: C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\id_name_foodb.csv
Food.json eerste paar regels:
    id       name
0  303      Bison
1  310  Wild boar
2  316    Buffalo
3  334    Chicken
4  353  Mule deer


In [3]:
# content.json filteren op basis van food_id's & relevante kolommen selecteren
content_data = []
with open(content_file_path, "r", encoding="utf-8") as content_file:
    for line in content_file:
        try:
            item = json.loads(line.strip())
            if item.get("food_id") == selected_food_id:  # Filter op de geselecteerde food_id
                # Alleen de gewenste kolommen opnemen
                filtered_item = {
                    "id": item.get("id"),
                    "food_id": item.get("food_id"),
                    "orig_food_id": item.get("orig_food_id"),
                    "orig_food_common_name": item.get("orig_food_common_name"),
                    "source_id": item.get("source_id"),
                    "orig_source_id": item.get("orig_source_id"),
                    "orig_source_name": item.get("orig_source_name"),
                    "source_type": item.get("source_type"),
                    "orig_content": item.get("orig_content"),
                    "orig_unit": item.get("orig_unit"),
                    "standard_content": item.get("standard_content"),
                }
                content_data.append(filtered_item)
        except json.JSONDecodeError as e:
            print(f"Fout bij het verwerken van een regel in content.json: {e}")

# Data omzetten naar een Pandas DataFrame
content_df = pd.DataFrame(content_data)
print("content_data eerste paar regels:")
print(content_df.head())

content_data eerste paar regels:
     id  food_id orig_food_id              orig_food_common_name  source_id  \
0  4690      506         0551      Beef brisket, soaked in brine          2   
1  4691      506         0551      Beef brisket, soaked in brine          2   
2  4692      506         0436  Beef, brisket, anterior part, raw          2   
3  4693      506         0436  Beef, brisket, anterior part, raw          2   
4  4694      506         0437    Beef, brisket, middle part, raw          2   

  orig_source_id  orig_source_name source_type orig_content orig_unit  \
0           0001    Protein, total    Nutrient      21000.0  mg/100 g   
1           0002  Protein, total-N    Nutrient       3400.0  mg/100 g   
2           0001    Protein, total    Nutrient      17600.0  mg/100 g   
3           0002  Protein, total-N    Nutrient       2800.0  mg/100 g   
4           0001    Protein, total    Nutrient      15900.0  mg/100 g   

  standard_content  
0          21000.0  
1          

id zal per definitie altijd een waarde hebben, daarnaast wordt er gefilterd op food_id waardoor dit ook altijd een waarde zal hebben. orig_food_id is de id van orig_food_common_name, nulwaarden hiervoor zijn dus niet problematisch zolang orig_food_common_name een waarde heeft. Deze worden gebruikt om binnenin de food_id een verdere distinctie te maken tussen verschillende onderdelen van het geheel, bv food_id 506 = beef & binnenin beef zijn er verschillende onderdelen zoals brisket, sirloin, minced meat, fat etc. orig_source_name & orig_source_id worden dan gebruikt om de verschillende componenten te kenmerken zoals glutamic acid, glucose etc. Source_type wordt gebruikt om een distinctie te maken tussen nutrients (FAT, PROTEIN, ...) & compounds (Glutamic acid, glucose, ...). Orig_content, orig_unit & standard_content worden gebruikt om de waarde van elke nutrient/compound te bewaren en quantificeren. Tenslotte blijft source_id over wat nog steeds een compleet mysterie voor me blijft, de naam lijkt belangrijk, maar ik kan geen enkele connectie vinden met een andere parameter en ik heb nog geen antwoord ontvangen van de eigenaars van de database.

In [4]:
missing_percentage = (content_df.isnull().mean() * 100)
print("\nPercentage ontbrekende waarden per kolom:")
print(missing_percentage)


Percentage ontbrekende waarden per kolom:
id                        0.000000
food_id                   0.000000
orig_food_id             36.174434
orig_food_common_name     0.200548
source_id                 0.000000
orig_source_id           40.297937
orig_source_name         40.297937
source_type               0.000000
orig_content             35.978935
orig_unit                35.978935
standard_content         35.978935
dtype: float64


In [5]:
# Sinds ontbrekende waardes in een van de content kolommen geen waardevolle datapunten zijn worden deze eruit gefilterd.
# Idem voor source_id & source_name kolom. Veranderen indien source_id toch een betekenis heeft in deze context!

# Controle 1: Correlatie tussen content kolommen
content_missing = content_df["orig_content"].isnull()
unit_missing = content_df["orig_unit"].isnull()
standard_missing = content_df["standard_content"].isnull()

# Controle of ontbrekende waarden in één van de content-kolommen altijd ontbreken in de andere twee
content_correlation = (content_missing == unit_missing) & (unit_missing == standard_missing)

if content_correlation.all():
    print("✅ Een nulwaarde in één van de content-kolommen (orig_content, orig_unit, standard_content) impliceert een nulwaarde in de andere twee.")
else:
    print("❌ Inconsistenties gevonden in de content-kolommen. Controleer de data.")

# Controle 2: Correlatie tussen source kolommen
source_name_missing = content_df["orig_source_name"].isnull()
source_id_missing = content_df["orig_source_id"].isnull()

# Controle of een nulwaarde in orig_source_name altijd gepaard gaat met een nulwaarde in orig_source_id
source_correlation = source_name_missing == source_id_missing

if source_correlation.all():
    print("✅ Een nulwaarde in orig_source_name impliceert altijd een nulwaarde in orig_source_id (en vice versa).")
else:
    print("❌ Inconsistenties gevonden in de source-kolommen. Controleer de data.")

# Aantal rijen vóór filtering
original_row_count = len(content_df)

# Verwijder rijen waar een van de kolommen nulwaarden bevat
columns_to_check = ["orig_content", "orig_unit", "standard_content", "orig_source_name", "orig_source_id"]
filtered_df = content_df.dropna(subset=columns_to_check, how="any")

# Aantal rijen na filtering
filtered_row_count = len(filtered_df)

# Bereken het percentage ontbrekende waarden opnieuw per kolom
missing_percentage = filtered_df.isnull().mean() * 100

# Toon het resultaat
print(f"\nAantal rijen oorspronkelijk: {original_row_count}")
print(f"Aantal rijen na filtering: {filtered_row_count}")
print(f"Aantal verwijderde rijen: {original_row_count - filtered_row_count}")
print("\nPercentage ontbrekende waarden per kolom na filtering:")
print(missing_percentage)

✅ Een nulwaarde in één van de content-kolommen (orig_content, orig_unit, standard_content) impliceert een nulwaarde in de andere twee.
✅ Een nulwaarde in orig_source_name impliceert altijd een nulwaarde in orig_source_id (en vice versa).

Aantal rijen oorspronkelijk: 138620
Aantal rijen na filtering: 82758
Aantal verwijderde rijen: 55862

Percentage ontbrekende waarden per kolom na filtering:
id                       0.0
food_id                  0.0
orig_food_id             0.0
orig_food_common_name    0.0
source_id                0.0
orig_source_id           0.0
orig_source_name         0.0
source_type              0.0
orig_content             0.0
orig_unit                0.0
standard_content         0.0
dtype: float64


In [6]:
# Bereken het aantal unieke waarden per kolom na filtering^
unique_values_per_column = content_df.apply(lambda col: col.nunique())
unique_values_per_column2 = filtered_df.apply(lambda col: col.nunique())

# Combineer de resultaten voor en na filtering in een DataFrame
comparison_df = pd.DataFrame({
    "Voor Filtering": unique_values_per_column,
    "Na Filtering": unique_values_per_column2
})

# Toon de side-by-side vergelijking
print("\nVergelijking van unieke waarden per kolom voor en na filtering:")
print(comparison_df)


Vergelijking van unieke waarden per kolom voor en na filtering:
                       Voor Filtering  Na Filtering
id                             138620         82758
food_id                             1             1
orig_food_id                      908           907
orig_food_common_name             908           907
source_id                       49997           139
orig_source_id                    241           240
orig_source_name                  205           204
source_type                         2             2
orig_content                    10720         10622
orig_unit                           6             6
standard_content                10154         10056


!!!controleer of dit verschil in orig_content tov standard content afhankelijk is van de source type of andere parameter, idem zoals hierboven!!!

In [7]:
import json
import tempfile
import os

json_data = filtered_df.to_dict(orient="records")
temp_json_path = os.path.join(tempfile.gettempdir(), "temp_json_view.json")

# Schrijf de JSON, met elk object op een aparte regel
with open(temp_json_path, "w", encoding="utf-8") as f:
    for record in json_data:
        json.dump(record, f, separators=(',', ':'), ensure_ascii=False)
        f.write("\n")  # Nieuwe regel voor elk object

# Open bestand in VS Code
os.system(f"code {temp_json_path}")

0

In [8]:
# Controle A: Simpele check of een `orig_source_name` meerdere `source_id`'s heeft
source_name_to_ids = filtered_df.groupby("orig_source_name")["source_id"].unique()
multiple_ids_per_name = source_name_to_ids[source_name_to_ids.apply(len) > 1]

total_unique_source_names = filtered_df["orig_source_name"].nunique()
print(f"\nTotaal aantal unieke orig_source_name's: {total_unique_source_names}")
print(f"Aantal orig_source_name met meerdere source_id's: {len(multiple_ids_per_name)}")

# Controle B: Welke `orig_source_name`'s horen bij een `source_id`?
df_multiple_names = (
    filtered_df.groupby(["source_id", "source_type"])["orig_source_name"]
    .unique()
    .reset_index()
)

# Enkel ID's behouden met meerdere namen
df_multiple_names = df_multiple_names[df_multiple_names["orig_source_name"].apply(len) > 1].copy()

if not df_multiple_names.empty:
    # Namen omzetten naar een string gescheiden door ';'
    df_multiple_names["orig_source_name"] = df_multiple_names["orig_source_name"].apply(lambda x: "; ".join(x))

    # Extra kolom toevoegen met het aantal namen
    df_multiple_names["Aantal Namen"] = df_multiple_names["orig_source_name"].apply(lambda x: x.count(";") + 1)

    # Opsplitsen in Nutrient en Compound en source_type verwijderen
    df_nutrient = df_multiple_names[df_multiple_names["source_type"] == "Nutrient"].drop(columns=["source_type"]).reset_index(drop=True)
    df_compound = df_multiple_names[df_multiple_names["source_type"] == "Compound"].drop(columns=["source_type"]).reset_index(drop=True)

    # Index laten starten vanaf 1
    df_nutrient.index += 1
    df_compound.index += 1

    # Aantal unieke ID's per categorie
    num_nutrient_ids = len(df_nutrient)
    num_compound_ids = len(df_compound)

    # HTML-weergave
    html_code = f"""
    <p><b>Totaal aantal unieke source_id's:</b> {filtered_df["source_id"].nunique()}</p>
    <p><b>Aantal source_id's met meerdere orig_source_name's:</b> {len(df_multiple_names)}</p>
    <p>- <b>Nutrient:</b> {num_nutrient_ids}</p>
    <p>- <b>Compound:</b> {num_compound_ids}</p>

    <div style="display: flex; justify-content: space-around;">
        <div>
            <h3>Source ID's met meerdere namen voor Nutrients</h3>
            {df_nutrient.to_html(index=True)}
        </div>
        <div>
            <h3>Source ID's met meerdere namen voor Compounds</h3>
            {df_compound.to_html(index=True)}
        </div>
    </div>
    """

    display(HTML(html_code))


Totaal aantal unieke orig_source_name's: 204
Aantal orig_source_name met meerdere source_id's: 0


,source_id,orig_source_name,Aantal Namen
1,2,"Protein, total; Protein, total-N; Protein",3
2,3,"Carbohydrates, total available; Carbohydrates, total; Carbohydrate, by difference",3
3,4,"Fat, total (Lipids); Total lipid (fat)",2
4,5,"Fiber, dietary; Fiber, total dietary (AOAC); Fiber, total dietary",3
5,31,C22:0; 22:0,2
6,36,"C22:5, n-3; 22:5 n-3 (DPA)",2
,source_id,orig_source_name,Aantal Namen
1,565,"Vitamin E, total; Vitamin E, alpha tocopherol; Vitamin E (alpha-tocopherol); Vitamin E, added",4
2,574,"Vitamin B6 (as pyridoxinehydrochloride), total; Vitamin B-6",2
3,753,"Alcohol (ethanol); Alcohol, ethyl",2


In [9]:
# Controle of orig_content gelijk is aan standard_content
equal_orig_standard = (filtered_df["orig_content"] == filtered_df["standard_content"])

# Percentage van gelijkheid
percentage_equal = equal_orig_standard.mean() * 100
print(f"\nPercentage waarin orig_content gelijk is aan standard_content: {percentage_equal:.2f}%")

# Rijen waar de waarden verschillen (indien nodig)
different_values = filtered_df[~equal_orig_standard]
print(f"\nAantal rijen waarin orig_content niet gelijk is aan standard_content: {len(different_values)}")
if not different_values.empty:
    print("\nVoorbeelden van verschillende waarden:")
    print(different_values[["id","orig_content", "standard_content"]].head())


Percentage waarin orig_content gelijk is aan standard_content: 97.89%

Aantal rijen waarin orig_content niet gelijk is aan standard_content: 1744

Voorbeelden van verschillende waarden:
           id   orig_content standard_content
13771  118472  280.831739962           1175.0
13772  118473  213.432122371            893.0
13773  118474   323.61376673           1354.0
13774  118475  315.009560229           1318.0
13775  118476  196.940726577            824.0


In [10]:
# Berekening voor unit_diff_percentage
unit_diff_percentage = filtered_df.groupby("orig_unit")["orig_content"].count().to_frame(name="total_count")
unit_diff_percentage["diff_count"] = filtered_df.groupby("orig_unit").apply(
    lambda x: (x["orig_content"] != x["standard_content"]).sum()
)
unit_diff_percentage["diff_percentage"] = (unit_diff_percentage["diff_count"] / unit_diff_percentage["total_count"]) * 100
unit_diff_percentage = unit_diff_percentage.sort_values("diff_percentage", ascending=False).head(10)

# Berekening voor source_diff_percentage
source_diff_percentage = filtered_df.groupby("source_type")["orig_content"].count().to_frame(name="total_count")
source_diff_percentage["diff_count"] = filtered_df.groupby("source_type").apply(
    lambda x: (x["orig_content"] != x["standard_content"]).sum()
)
source_diff_percentage["diff_percentage"] = (source_diff_percentage["diff_count"] / source_diff_percentage["total_count"]) * 100
source_diff_percentage = source_diff_percentage.sort_values("diff_percentage", ascending=False)

# Berekening van de overlap tussen orig_unit en source_type
diff_df = filtered_df[filtered_df["orig_content"] != filtered_df["standard_content"]]
relevant_units = ["kcal/100 g", "mg/100 g"]
filtered_diff_df = diff_df[diff_df["orig_unit"].isin(relevant_units)]

total_differences_per_unit = filtered_diff_df["orig_unit"].value_counts()

expected_mapping = {
    "kcal/100 g": "Nutrient",
    "mg/100 g": "Compound"
}

matching_counts = filtered_diff_df[
    filtered_diff_df.apply(lambda row: row["source_type"] == expected_mapping.get(row["orig_unit"], ""), axis=1)
]["orig_unit"].value_counts()

overlap_percentage = (matching_counts / total_differences_per_unit) * 100

# Opmaak van de outputtekst
overlap_text = "<br>".join([f"• {unit}: {percent:.1f}% van de verschillen komen overeen met source_type {expected_mapping[unit]}" 
                            for unit, percent in overlap_percentage.fillna(0).items()])

# Zet de tabellen en overlappercentage in HTML naast elkaar en onder elkaar
html_code = f"""
<div style="display: flex; justify-content: space-around;">
    <div>{unit_diff_percentage.to_html()}</div>
    <div>{source_diff_percentage.to_html()}</div>
</div>
<div style="margin-top: 20px; font-size: 14px; font-weight: bold;">
    <p>Overlap percentages:</p>
    <p>{overlap_text}</p>
</div>
"""

display(HTML(html_code))

C:\Users\bramd_finhsgu\AppData\Local\Temp\ipykernel_25664\3303536413.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unit_diff_percentage["diff_count"] = filtered_df.groupby("orig_unit").apply(
C:\Users\bramd_finhsgu\AppData\Local\Temp\ipykernel_25664\3303536413.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  source_diff_percentage["diff_count"] = filtered_df.groupby("source_type").apply(


,total_count,diff_count,diff_percentage
orig_unit,,,
kcal/100 g,1767,907,51.329938
mg/100 g,78759,837,1.062736
NE,47,0,0.000000
IU,1260,0,0.000000
RE,878,0,0.000000
α-TE,47,0,0.000000
,total_count,diff_count,diff_percentage
source_type,,,
Nutrient,15551,907,5.832422


In [ ]:
# Opslaan dataset na het filteren

# Definieer de folder waar je het bestand wilt opslaan
output_folder = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\foodb_filtered_datasets"

# Maak de volledige bestandsnaam dynamisch met de food_id
output_filename = f"filtered_data_food_id_{selected_food_id}.csv"

# Combineer folder en bestandsnaam
output_path = f"{output_folder}\\{output_filename}"

# Sla het gefilterde DataFrame op als CSV-bestand
filtered_df.to_csv(output_path, index=False)

# Bevestiging van opslaan
print(f"Bestand opgeslagen op: {output_path}") 

In [11]:
# Unieke combinaties van food name + food ID ophalen
unique_foods = filtered_df.drop_duplicates(subset=["orig_food_common_name", "orig_food_id"])[["orig_food_common_name", "orig_food_id"]].values

# Set met vleesdelen (hoofdletterongevoelig)
meat_parts = {"chuck", "rib", "short loin", "sirloin", "round", "brisket", "neck", "mince", "minced", "shoulder", 
              "tenderloin", "meat", "topside", "loin", "shank", "plate", "flank", "ribeye", "oxtail", "tongue", 
              "rump", "entrecote", "fillet", "striploin", "t-bone", "strip steak", "breast"}

# Set met woorden die wijzen op bereiding (hoofdletterongevoelig)
cooked_terms = {"cooked", "stewed", "boiled", "grilled", "roasted", "fried", "smoked", "braised", "cured", 
                "brain", "kidney", "liver", "lungs", "heart", "pancreas", "thymus", "spleen", "suet", "tripe"}

def filter_meat_parts(food_list, meat_parts, cooked_terms):
    """
    Filtert unieke vleesdelen en bijbehorende 'orig_food_id'.
    
    Args:
        food_list (list): Lijst met unieke (food_name, food_id) paren.
        meat_parts (set): Set met relevante vleesstukken.
        cooked_terms (set): Set met woorden die op bereiding wijzen.

    Returns:
        tuple: (meat_matches, non_meat_matches)
    """
    meat_matches = {}  # Vleesdeel → food_id
    non_meat_matches = {}  # Niet-vlees of vlees met kooktermen → food_id

    for food_name, food_id in food_list:
        food_name_lower = food_name.lower()

        if any(part in food_name_lower for part in meat_parts):  # Is het een vleesdeel?
            if any(term in food_name_lower for term in cooked_terms):  # Bevat het kooktermen?
                non_meat_matches[food_name] = food_id
            else:
                meat_matches[food_name] = food_id  # Opslaan als uniek vleesdeel
        else:
            non_meat_matches[food_name] = food_id

    return meat_matches, non_meat_matches

# Filter toepassen op unieke combinaties
meat_matches, non_meat_matches = filter_meat_parts(unique_foods, meat_parts, cooked_terms)

# ✅ Verbeterde printregels:
print(f"Er zijn {len(meat_matches)} unieke vleesstukken gevonden en {len(non_meat_matches)} niet-gematchte items (niet-vlees of vlees met kooktermen).\n")

print("**Gevonden vleesstukken met bijbehorende orig_food_id:**")
for food, food_id in meat_matches.items():
    print(f"[{food_id}] {food}")  # Unieke ID + vleesnaam tonen

print("\n**Niet-gematchte items met bijbehorende orig_food_id:**")
for food, food_id in non_meat_matches.items():
    print(f"[{food_id}] {food}")  # Unieke ID + niet-gematchte items tonen


Er zijn 317 unieke vleesstukken gevonden en 590 niet-gematchte items (niet-vlees of vlees met kooktermen).

**Gevonden vleesstukken met bijbehorende orig_food_id:**
[0551] Beef brisket, soaked in brine
[0436] Beef, brisket, anterior part, raw
[0437] Beef, brisket, middle part, raw
[0438] Beef, brisket, posterior part, raw
[0432] Beef, chuck, raw
[0430] Beef, entrecote "cap on", raw
[0422] Beef, eye of round, raw
[0428] Beef, fillet, defatted, raw
[0440] Beef, flank, raw
[0439] Beef, flatribs, raw
[1342] Beef, low fat, minced, raw
[0201] Beef, meat, 5-10 % fat, raw
[0202] Beef, meat, approx. 15 % fat, raw
[0200] Beef, meat, approx. 25 % fat, raw
[0199] Beef, meat, lean (<5 % fat), raw
[1296] Beef, mince, raw
[0433] Beef, neck, raw
[0420] Beef, outside, round, raw
[0423] Beef, rump, raw
[0425] Beef, rumpsteak "cap off", raw
[0424] Beef, rumpsteak "cap on", raw
[0417] Beef, shin + leg (shank), raw
[0434] Beef, shoulder, dorsal part, raw
[0435] Beef, shoulder, ventral part
[0427] Beef, sir

In [12]:
meat_ids = set(meat_matches.values())  # Zet alle vlees-ID’s in een set voor snelle lookup
filtered_df_meat = filtered_df[filtered_df["orig_food_id"].astype(str).isin(meat_ids)]

original_count = len(filtered_df)
filtered_count = len(filtered_df_meat)

print(f"Aantal rijen voor filtering: {original_count}")
print(f"Aantal vleesproducten verwijderd: {original_count - filtered_count}")
print(f"Aantal rijen na filtering: {filtered_count}")



# Splits filtered_df_meat op basis van source_type
df_meat_nutrient = filtered_df_meat[filtered_df_meat["source_type"] == "Nutrient"].copy()
df_meat_compound = filtered_df_meat[filtered_df_meat["source_type"] == "Compound"].copy()

# Print aantal rijen in elke dataframe
print(f"Aantal rijen in df_meat_nutrient: {len(df_meat_nutrient)}")
print(f"Aantal rijen in df_meat_compound: {len(df_meat_compound)}")
print(df_meat_nutrient)

Aantal rijen voor filtering: 82758
Aantal vleesproducten verwijderd: 52812
Aantal rijen na filtering: 29946
Aantal rijen in df_meat_nutrient: 5626
Aantal rijen in df_meat_compound: 24320
           id  food_id orig_food_id                    orig_food_common_name  \
0        4690      506         0551            Beef brisket, soaked in brine   
1        4691      506         0551            Beef brisket, soaked in brine   
2        4692      506         0436        Beef, brisket, anterior part, raw   
3        4693      506         0436        Beef, brisket, anterior part, raw   
4        4694      506         0437          Beef, brisket, middle part, raw   
...       ...      ...          ...                                      ...   
15529  120230      506        36011   T.G.I. FRIDAY'S, classic sirloin steak   
15530  120231      506        36013  Restaurant, family style, sirloin steak   
15531  120232      506        36013  Restaurant, family style, sirloin steak   
15534  120235

In [13]:
import os

# Bereken unieke Nutrients en Compounds vóór filtering
nutrient_count_before = filtered_df[filtered_df["source_type"] == "Nutrient"][["orig_source_name", "source_id"]].drop_duplicates().shape[0]
compound_count_before = filtered_df[filtered_df["source_type"] == "Compound"][["orig_source_name", "source_id"]].drop_duplicates().shape[0]

# Bereken unieke Nutrients en Compounds na filtering
nutrient_count_after = df_meat_nutrient[["orig_source_name", "source_id"]].drop_duplicates().shape[0]
compound_count_after = df_meat_compound[["orig_source_name", "source_id"]].drop_duplicates().shape[0]

# Print aantal unieke entries vóór en na filtering
print(f"Aantal unieke Nutrients vóór filtering: {nutrient_count_before}, en na filtering: {nutrient_count_after}.")
print(f"Aantal unieke Compounds vóór filtering: {compound_count_before}, en na filtering: {compound_count_after}.")

# Maak unieke DataFrames voor Nutrients en Compounds na filtering
df_nutrient_meat = df_meat_nutrient[["orig_source_name", "source_id"]].drop_duplicates().reset_index(drop=True)
df_compound_meat = df_meat_compound[["orig_source_name", "source_id"]].drop_duplicates().reset_index(drop=True)

# Opslaan als CSV-bestanden
output_dir = "output"  # Opslagmap
os.makedirs(output_dir, exist_ok=True)  # Maak map aan als die nog niet bestaat

nutrient_csv_path = os.path.join(output_dir, "df_nutrient_meat.csv")
compound_csv_path = os.path.join(output_dir, "df_compound_meat.csv")

# Controleer of DataFrames leeg zijn vóór opslaan
if not df_nutrient_meat.empty:
    df_nutrient_meat.to_csv(nutrient_csv_path, index=False, encoding="utf-8", sep=";")
    print(f"Nutrient CSV opgeslagen: {os.path.abspath(nutrient_csv_path)}")
else:
    print("⚠️ df_nutrient_meat is leeg en werd niet opgeslagen.")

if not df_compound_meat.empty:
    df_compound_meat.to_csv(compound_csv_path, index=False, encoding="utf-8", sep=";")
    print(f"Compound CSV opgeslagen: {os.path.abspath(compound_csv_path)}")
else:
    print("⚠️ df_compound_meat is leeg en werd niet opgeslagen.")

Aantal unieke Nutrients vóór filtering: 43, en na filtering: 42.
Aantal unieke Compounds vóór filtering: 161, en na filtering: 158.
Nutrient CSV opgeslagen: c:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_scripts\Masterthesis_2425_Bram_Duthoo\scripts\output\df_nutrient_meat.csv
Compound CSV opgeslagen: c:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_scripts\Masterthesis_2425_Bram_Duthoo\scripts\output\df_compound_meat.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Functie om de data-exploratie uit te voeren
def explore_nutrient_data(df_meat_nutrient, exclude_ids=[]):
    # Verwijder filteringstap: originele data wordt gebruikt zonder initiële filtering
    df_filtered = df_meat_nutrient[~df_meat_nutrient['source_id'].isin(exclude_ids)]
    
    # Unieke nutrienten met bijbehorende eenheden controleren
    nutrient_units = df_filtered.groupby('source_id')['orig_unit'].unique()

    # Lijst om inconsistenties in eenheden bij te houden
    inconsistent_units = []

    # Maak een submap voor het opslaan van de figuren
    output_dir = "nutrients"
    os.makedirs(output_dir, exist_ok=True)

    # Lijst voor het samenvatten van de distributies van nutrienten
    summary_data = []

    # Plot per nutrient
    for nutrient_id, units in nutrient_units.items():
        # Als er meer dan 1 unieke unit is per nutrient, slaan we de plot over en bewaren we de ID
        if len(units) > 1:
            inconsistent_units.append((nutrient_id, units))
            continue
        
        # Filter de data voor dit specifieke nutrient
        nutrient_data = df_filtered[df_filtered['source_id'] == nutrient_id].copy()
        
        # Controleer op niet-numerieke waarden en probeer deze te converteren
        nutrient_data['orig_content'] = pd.to_numeric(nutrient_data['orig_content'], errors='coerce')

        # Filter: verwijder NaN-waarden en negatieve waarden
        nutrient_data = nutrient_data.dropna(subset=['orig_content'])
        nutrient_data = nutrient_data[nutrient_data['orig_content'] >= 0]  # Behoud alleen positieve waarden

        # Als er geen data overblijft na filtering of slechts 1 unieke waarde heeft, sla deze nutrient over
        if nutrient_data.empty or nutrient_data['orig_content'].nunique() == 1:
            continue
        
        # Bereken de sample size
        sample_size = len(nutrient_data)
        
        # Als de sample size kleiner dan 25 is, werk met medianen en IQR, anders maak een distributieplot
        if sample_size < 25:
            median = nutrient_data['orig_content'].median()
            iqr = nutrient_data['orig_content'].quantile(0.75) - nutrient_data['orig_content'].quantile(0.25)
            plt.figure(figsize=(8, 6))
            plt.boxplot(nutrient_data['orig_content'], vert=False)
            plt.title(f'Nutriënt: {nutrient_data["orig_source_name"].iloc[0]} (n={sample_size})')
            plt.xlabel(f'Value ({units[0]})')
            plt.ylabel('Frequentie')
            plt.text(median, 1.1, f'Median: {median:.2f}\nIQR: {iqr:.2f}', horizontalalignment='center')
            plt.savefig(os.path.join(output_dir, f"boxplot_{nutrient_id}.png"))
            plt.close()
        else:
            # Plot de distributie met verbeterde binning
            plt.figure(figsize=(8, 6))
            sns.histplot(nutrient_data['orig_content'], kde=True, bins=min(10, sample_size//2))
            plt.title(f'Distributie van nutrient: {nutrient_data["orig_source_name"].iloc[0]} (n={sample_size})')
            plt.xlabel(f'Value ({units[0]})')  # Gebruik de enige unit
            plt.ylabel('Frequentie')
            plt.savefig(os.path.join(output_dir, f"distribution_{nutrient_id}.png"))
            plt.close()

        # Voeg deze nutrient toe aan de samenvattende data
        summary_data.append({
            'nutrient_name': nutrient_data['orig_source_name'].iloc[0],
            'nutrient_id': nutrient_id,
            'unit': units[0],
            'sample_size': sample_size,
            'mean': round(nutrient_data['orig_content'].mean(), 2),
            'std': round(nutrient_data['orig_content'].std(), 2),
            'min': round(nutrient_data['orig_content'].min(), 2),
            'max': round(nutrient_data['orig_content'].max(), 2)
        })
    
    # Als er inconsistente units zijn, geef dit aan
    if inconsistent_units:
        print("⚠️ Er zijn inconsistenties in de eenheden voor de volgende nutrients:")
        for nutrient_id, units in inconsistent_units:
            print(f"Nutriënt ID {nutrient_id}: Eenheden - {units}")
        print("De distributies voor deze nutrienten worden niet weergegeven.")
    
    # Samenvattende DataFrame maken
    summary_df = pd.DataFrame(summary_data)

    # Opslaan van samenvattende DataFrame als CSV
    summary_csv_path = os.path.join(output_dir, "nutrient_summary.csv")
    summary_df.to_csv(summary_csv_path, index=False, sep=";")

    print(f"Samenvattende data opgeslagen als CSV: {summary_csv_path}")
    
    return summary_df, inconsistent_units

# Voorbeeld van het aanroepen van de functie met df_meat_nutrient en optioneel exclude_ids
exclude_ids = [8, 10, 15, 34]  # Vervang door de daadwerkelijke IDs die je wilt uitsluiten
summary_df, inconsistent_units = explore_nutrient_data(df_meat_nutrient, exclude_ids)

# Samenvatting van de nutrienten die problemen hadden met inconsistentie in units
print("\nInconsistente eenheden:")
for nutrient_id, units in inconsistent_units:
    print(f"Nutriënt ID {nutrient_id} heeft de volgende eenheden: {units}")